In [20]:
Model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=GOOGLE_API_KEY)
Model.invoke('What is 2+2?')

AIMessage(content='2 + 2 = 4', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a781841d-7510-47f2-b996-66752a74d972-0', usage_metadata={'input_tokens': 8, 'output_tokens': 7, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})

In [21]:
class BlogState(TypedDict):
    title:str
    outline :str
    content:str

In [22]:
def genrate_outline(state:BlogState)->BlogState:
    title = state['title']
    prompt =f"""Your are a helpfull assistant based on the title genrate the detailed blog outline.\n{title}"""
    outline=Model.invoke(prompt)

    state['outline'] = outline
    return state


In [23]:
def genrate_blog(state:BlogState)->BlogState:
    outline = state['outline']
    prompt = f"""Based on this detailed on the title {state['title']} and on outline genrate the blog content for me it should be relevent concise and to the point-outlines is \n{outline}"""
    content = Model.invoke(prompt)
    state['content']=content
    return state

In [24]:
from langgraph.graph import StateGraph,START,END
Graph =  StateGraph(BlogState)

In [25]:
# add edges and nodes
Graph.add_node("genrate_outline",genrate_outline)
Graph.add_node("genrate_blog",genrate_blog)

# add edhes
Graph.add_edge(START,"genrate_outline")
Graph.add_edge("genrate_outline","genrate_blog")
Graph.add_edge("genrate_blog",END)


workflow = Graph.compile()

In [26]:
from IPython.display import display, Image

try:
    img_data = workflow.get_graph().draw_mermaid_png()
    display(Image(data=img_data))
except Exception as e:
    print(f"Problem: There was a problem: {e}")


Problem: There was a problem: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [27]:
intial_state = {"title":"quantum physics"}
result = workflow.invoke(intial_state)
print(result)

{'title': 'quantum physics', 'outline': AIMessage(content='Okay, this is a fantastic topic! Quantum Physics is inherently fascinating, mind-bending, and crucial to our understanding of the universe. Here\'s a detailed blog outline designed to engage a broad audience, from curious beginners to those with a basic science background.\n\n---\n\n## Blog Title: Decoding the Quantum Realm: A Journey into the Universe\'s Weirdest Physics\n\n**Meta Description:** Dive into the mind-bending world of Quantum Physics! Explore its core concepts, famous experiments, real-world applications, and the mysteries that continue to challenge our understanding of reality.\n\n---\n\n### I. Introduction: Welcome to the Quantum Wonderland\n\n*   **A. The Hook: Beyond Everyday Reality**\n    *   Start with a statement about how our everyday experience of the world differs dramatically from what happens at the smallest scales.\n    *   "What if everything you thought you knew about reality was just a macroscopic